In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd

pd.set_option("display.max_columns", None)

In [2]:
#libraries 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import pendulum
from PIL import Image
import tensorflow as tf

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import ConvLSTM2D
from keras.layers import BatchNormalization


In [4]:
from keras.layers import Conv2D, GlobalAveragePooling2D

In [5]:
#load data 
data = pd.read_pickle('/Users/lmoh/Downloads/UT_Smoking_Data_pickles/Participant1_Data.pickle')
data.head()

,timestamp_WD,Accelerometer_x_WD,Accelerometer_y_WD,Accelerometer_z_WD,Linear_acceleration_sensor_x_WD,Linear_acceleration_sensor_y_WD,Linear_acceleration_sensor_z_WD,Class_label
0,2016-02-12 11:57:51.218000+00:00,-5.973572,-6.710114,2.555145,-0.474457,0.382236,0.345239,SmokeSD
1,2016-02-12 11:57:51.238000+00:00,-5.980713,-6.838654,2.605133,-0.471420,0.249793,0.383285,SmokeSD
2,2016-02-12 11:57:51.258000+00:00,-6.025940,-6.895783,2.514679,-0.504273,0.187889,0.285115,SmokeSD
3,2016-02-12 11:57:51.279000+00:00,-5.956909,-7.031464,2.752716,-0.424346,0.048824,0.503679,SmokeSD
4,2016-02-12 11:57:51.298000+00:00,-5.959290,-6.900543,2.743195,-0.414628,0.176953,0.475054,SmokeSD


In [6]:
data.shape

(904400, 8)

In [7]:
data['Class_label'].unique()

array(['SmokeSD', 'SmokeST', 'Eat', 'DrinkSD', 'DrinkST', 'Sit', 'Stand'],
      dtype=object)

In [8]:
num_classes = 7
#time_steps = time_steps
#input_shape = X_train_res.shape[1:]
#n_neurons= n_neurons

In [9]:
#Some functions to build a ResNet 34 model, if anyone wants to try it
#the identity block function is missing, so are some calling functions will add later


In [10]:
#convolutional block for ResNet function
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x


In [11]:
#ResNet34 model function
def ResNet34(shape = (32, 32, 3), classes = 10):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

In [13]:
#pretrained transfer learning models from keras
#ResNet50
resnet_model = Sequential()
pretrained_model = tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=7,
    
)
#making sure the already trained model doesn't train some more uselessly/ simplifies fitting process later
for layer in pretrained_model.layers: 
    layer.trainable = False 
#adding the pretrained model to own model   
resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(64), activation='relu')
resnet_model.add(Dense(5), activation='softmax')

ValueError: If using `weights` as `"imagenet"` with `include_top` as true, `classes` should be 1000

In [14]:
#ResNet 101V2
resnet_model100 = Sequential()
pretrained_model100 = tf.keras.applications.ResNet101V2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=7,
    classifier_activation="softmax",
)
#making sure the already trained model doesn't train some more uselessly/ simplifies fitting process later
for layer in pretrained_model.layers: 
    layer.trainable = False 
#adding the pretrained model to own model   
resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(64), activation='relu')
resnet_model.add(Dense(5), activation='softmax')

ValueError: If using `weights` as `"imagenet"` with `include_top` as true, `classes` should be 1000

In [15]:
from tensorflow.keras.layers import Layer, Activation, Add
from tensorflow.keras.models import Model
import tensorflow as tf

In [16]:
segments.shape

NameError: name 'segments' is not defined

In [18]:
#Coding ResNet from scratch, as seen in the paper 
class ResNet18(Model):
    def __init__(self,):
        super(ResNet18, self).__init__(name = 'resnet_18')
        
        #conv layer
        self.conv_1 = CustomConv2D(64, 7, 2, padding='same')
        self.batch_norm = BatchNormalization(axis=3)
        self.max_pool = MaxPooling2D((3, 2))
        
        #Res layers
        self.conv_2_1 = ResidualBlock(64, 2)
        self.conv_2_2 = ResidualBlock(64)
        
        self.conv_3_1 = ResidualBlock(128, 2)
        self.conv_3_2 = ResidualBlock(128)
        
        self.conv_4_1 = ResidualBlock(256, 2)
        self.conv_4_2 = ResidualBlock(256)
        
        self.conv_5_1 = ResidualBlock(512, 2)
        self.conv_5_2 = ResidualBlock(512)
        
        #Global average pooling / Flatten / Dense / Output layer 
        
        self.global_pool = GlobalAveragePooling2D()
        self.flatten = Flatten()
        self.dense_32 = Dense(32, activation='relu')
        self.dense_7 = Dense(7, activation='softmax')
        
    def call(self, x): 
        #conv layer
        x = self.conv_1(x)
        x = self.batch_norm(x)
        x = self.max_pool(x)
        
        #res layers
        x = self.conv_2_1(x)
        x = self.conv_2_2(x)
        
        x = self.conv_3_1(x)
        x = self.conv_3_2(x)
        
        x = self.conv_4_1(x)
        x = self.conv_4_2(x)
        
        x = self.conv_5_1(x)
        x = self.conv_5_2(x)
        
        #output shenanigans 
        x = self.global_pool(x)
        x = self.flatten(x)
        x = self.dense_32(x)
        
        return self.dense_7          

In [19]:
 class ResidualBlock(Layer):
        def __init__(self, n_channels, n_strides=1):
            super(ResidualBlock, self).__init__(name = 'res block')
            
            self.dotted = (n_strides != 1)
            
            self.custom_conv_1= CustomConv2D(n_channels, 3, n_strides, padding="same")
            self.batch_norm1 = BatchNormalization(axis=3)
            self.custom_conv_2= CustomConv2D(n_channels, 3, 1, padding="same")
            self.batch_norm2 = BatchNormalization(axis=3)
            
            self.activation = Activation('relu')
            if self.dotted:
                self.custom_conv_3 = CustomConv2D(n_channels, 1, n_strides)
            #reminder to add Squeeze and Excitation block (?)
        
        def call(self, input):
            x = self.custom_conv_1(input)
            x = self.batch_norm1(x)
            x = self.custom_conv_2(x)
            x = self.batch_norm2(x)
            
            if self.dotted:
                x_add = self.custom_conv_3(input)
                x_add = Add()([x, x_add])
            else: 
                    x_add = Add()([x, input])
                    
            return self.activation(x_add)
    
            

In [20]:
#run this cell first
class CustomConv2D(Layer):
    def __init__(self, n_filters, kernel_size, n_strides, padding="same"):
        super(CustomConv2D, self).__init__(name = 'custom_conv2d')
        self.conv = Conv2D(filters = n_filters,
                           kernel_size = kernel_size,
                           activation='relu', 
                           strides=n_strides, 
                           padding=padding)
            
            #self.batch_norm = BatchNormalization()
        
    def call(self, x, training):
        x = self.conv(x)
       #x = self.batch_norm(x)
            
        return x

In [21]:
resnet_18 = ResNet18()
resnet_18(tf.zeros([1, 30, 3]))
resnet_18.summary()

ValueError: Exception encountered when calling layer 'custom_conv2d' (type CustomConv2D).

Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (1, 30, 3)

Call arguments received by layer 'custom_conv2d' (type CustomConv2D):
  • x=tf.Tensor(shape=(1, 30, 3), dtype=float32)
  • training=None

In [26]:
from scipy import stats

In [27]:
#segmentation function

def create_segments_and_labels(df, time_steps, step):

    # x, y, z acceleration as features
    n_features = 3
    # Each generated sequence contains 200 training examples
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        xs = df['Accelerometer_x_WD'].values[i: i + time_steps]
        ys = df['Accelerometer_y_WD'].values[i: i + time_steps]
        zs = df['Accelerometer_z_WD'].values[i: i + time_steps]
        # Retrieve the most often used label in each segment
        label = stats.mode(df['Class_label'][i: i + time_steps])[0][0]
        segments.append([xs, ys, zs])
        labels.append(label)

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels =np.asarray(pd.get_dummies(labels), dtype = np.float32)

    return reshaped_segments, labels


In [30]:
segments, labels = create_segments_and_labels(data, 500, 500)

/var/folders/06/bsdxkm7j1tnclcj0g97ll0cw0000gn/T/ipykernel_26045/1174174176.py:15: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  label = stats.mode(df['Class_label'][i: i + time_steps])[0][0]
/var/folders/06/bsdxkm7j1tnclcj0g97ll0cw0000gn/T/ipykernel_26045/1174174176.py:15: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  label = stats.mode(df['Class_label'][i: i + time_steps])[0][0]


In [33]:
segments = segments.reshape(1808, 500, 1, 3)

In [31]:
segments.shape

(1808, 500, 3)

In [35]:
#compiling model, no train test split
resnet_18.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

In [36]:
resnet_18.fit(segments, labels, epochs=30)


Epoch 1/30


ValueError: in user code:

    File "/Users/lmoh/.pyenv/versions/3.10.6/envs/mocap/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/lmoh/.pyenv/versions/3.10.6/envs/mocap/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/lmoh/.pyenv/versions/3.10.6/envs/mocap/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/lmoh/.pyenv/versions/3.10.6/envs/mocap/lib/python3.10/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/Users/lmoh/.pyenv/versions/3.10.6/envs/mocap/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/va